## Introduction
In this kernel, I'd like to share the approach using 1D Convolutional Neural Networks(1D CNN). 1D CNN is sometimes effective to analyze time series data. This kernel introduces U-Net architecture with 1D CNN.

## Import Library

In [ ]:
import os
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import scipy as sp
import itertools
import warnings
import time
import pywt
import random as rn
from numpy.fft import *
import pyarrow.parquet as pq

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Reshape, Conv1D, BatchNormalization, Activation, AveragePooling1D, GlobalAveragePooling1D, Lambda, Input, Concatenate, Add, UpSampling1D, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.callbacks import Callback

from tensorflow.keras.callbacks import  ModelCheckpoint
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from tensorflow.keras.layers import Conv2D, Concatenate, MaxPooling2D
from tensorflow.keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import constraints
# from keras import backend as K
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ZeroPadding2D
import tensorflow.keras.callbacks as callbacks
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import multiply

from tensorflow.keras import optimizers

from tensorflow.keras.layers import Input,Dropout,BatchNormalization,Activation,add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute
from tensorflow.keras.optimizers import SGD


## Load and Split Dataset
Simply split the input data into certain length.

In [ ]:
# def filter_signal(signal, threshold=1e7):
#     fourier = rfft(signal)
# #     print(fourier.shape)
#     frequencies = rfftfreq(signal.size, d=20e-6/signal.size)
# #     print(frequencies.shape)
#     fourier[frequencies > threshold] = 0
#     return irfft(fourier)

In [ ]:
# signal = np.array([[2, 3, 1], [4, 5, 6], [8, 2, 3], [2, 5, 6]])
# print(filter_signal(signal))

In [ ]:
df_train = pd.read_csv("../input/liverpool-ion-switching/train.csv")
df_test = pd.read_csv("../input/liverpool-ion-switching/test.csv")

# print(df_train.shape)

# I don't use "time" feature
# train_input0 = filter_signal(df_train["signal"].values)
# train_input = train_input0.reshape(-1, 4000, 1)
# plt.figure(figsize=(15, 10))
# plt.plot(df_train["signal"], label='Raw')
# plt.plot(train_input0, label='Filtered')
# plt.show()
train_input = df_train["signal"].values.reshape(-1,4000,1)#number_of_data:1250 x time_step:4000
print(train_input.shape)
train_input_mean = train_input.mean()
train_input_sigma = train_input.std()
train_input = (train_input-train_input_mean)/train_input_sigma
print(df_test["signal"].values.shape)
test_input = df_test["signal"].values.reshape(-1,4000,1)
test_input = (test_input-train_input_mean)/train_input_sigma

train_target = pd.get_dummies(df_train["open_channels"]).values.reshape(-1,4000,11)
idx = np.arange(train_input.shape[0])
train_idx, val_idx = train_test_split(idx, random_state = 111,test_size = 0.2)
# print(train_idx[:20])
val_input = train_input[val_idx]
# val_input = filter_signal(val_input)
train_input = train_input[train_idx] 
# train_input = filter_signal(train_input)
val_target = train_target[val_idx]
train_target = train_target[train_idx] 

print("train_input:{}, val_input:{}, train_target:{}, val_target:{}".format(train_input.shape[0], val_input.shape, train_target.shape, val_target.shape))

## Define Model
This section defines U-Net(se-resnet base).
Input and output of the U-Net are follows:
* Input: 4000 time steps of "signal"
* Output: 4000 time steps of "open_channels"

In [ ]:
# def convolution_block(x, filters, size, strides=1, padding='same', activation=True):
#     x = Conv1D(filters, size, strides=strides, padding=padding)(x)
#     x = BatchNormalization()(x)
#     if activation == True:
#         x = LeakyReLU(alpha=0.1)(x)
#     return x

# def residual_block(blockInput, num_filters=16):
#     x = LeakyReLU(alpha=0.1)(blockInput)
#     x = BatchNormalization()(x)
#     blockInput = BatchNormalization()(blockInput)
#     x = convolution_block(x, num_filters, 7 )
#     x = convolution_block(x, num_filters, 7, activation=False)
#     x = Add()([x, blockInput])
#     return x

# def upsample(x):
#     u = UpSampling1D(10)(x)
#     return u

# def downsample(x):
#     x = AveragePooling1D(10)(x)
#     return x

# # a is conv in same layer
# # d is downsample
# # u is upsample
# def DenseHRNet(input_shape=(None, 1)):
#     layer_n = 64
#     kernel_size = 7
#     depth = 2
    
#     input_layer = Input(input_shape)
#     x = input_layer
#     for i in range(depth):
#         x = residual_block(x)
#     l1_1 = x
#     l1_2 = residual_block(l1_1)
#     l1_3 = residual_block(l1_2)
    
#     l2_1 = downsample(l1_1)
#     l2_2 = residual_block(l2_1)
    
#     a1_4 = residual_block(l1_3)
#     u2_2 = upsample(l2_2)
#     l1_4 = Concatenate()([a1_4, u2_2])
    
#     l1_5 = residual_block(l1_4, num_filters=32)
    
#     d1_3 = downsample(l1_4)
#     a2_2 = residual_block(l2_2)
#     l2_3 = Concatenate()([d1_3, a2_2]) 
    
#     l2_4 = residual_block(l2_3, num_filters=48)
    
#     d1_5 = downsample(l1_5)
#     a2_4 = residual_block(l2_4, num_filters=48)
#     l2_5 = Concatenate()([d1_5, a2_4])
    
#     a1_5 = residual_block(l1_5, num_filters=32)
#     u2_4 = upsample(l2_4)
#     u2_5 = upsample(l2_5)
#     l1_6 = Concatenate()([a1_5, u2_4, u2_5])
    
#     x = Conv1D(11, kernel_size=kernel_size, strides=1, padding="same")(x)
#     out = Activation("softmax")(x)
#     model = Model(input_layer, out)
#     return model




# def augmentations(input_data, target_data):
#     #flip
#     if np.random.rand()<0.5:    
#         input_data = input_data[::-1]
#         target_data = target_data[::-1]

#     return input_data, target_data


# def Datagen(input_dataset, target_dataset, batch_size, is_train=False):
#     x=[]
#     y=[]
  
#     count=0
#     idx_1 = np.arange(len(input_dataset))
#     #idx_2 = np.arange(len(input_dataset))
#     np.random.shuffle(idx_1)
#     #np.random.shuffle(idx_2)

#     while True:
#         for i in range(len(input_dataset)):
#             input_data = input_dataset[idx_1[i]]
#             target_data = target_dataset[idx_1[i]]
#             #input_data_mix = input_dataset[idx_2[i]]
#             #target_data_mix = target_dataset[idx_2[i]]

#             if is_train:
#                 input_data, target_data = augmentations(input_data, target_data)
#                 #input_data_mix, target_data_mix = augmentations(input_data_mix, target_data_mix)
                
#             x.append(input_data)
#             y.append(target_data)
#             count+=1
#             if count==batch_size:
#                 x=np.array(x, dtype=np.float32)
#                 y=np.array(y, dtype=np.float32)
#                 inputs = x
#                 targets = y       
#                 x = []
#                 y = []
#                 count=0
#                 yield inputs, targets

# class macroF1(Callback):
#     def __init__(self, model, inputs, targets):
#         self.model = model
#         self.inputs = inputs
#         self.targets = np.argmax(targets, axis=2).reshape(-1)

#     def on_epoch_end(self, epoch, logs):
#         pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
#         f1_val = f1_score(self.targets, pred, average="macro")
#         print("val_f1_macro_score: ", f1_val)
        
# def model_fit(model, train_inputs, train_targets, val_inputs, val_targets, n_epoch, batch_size=32):
#     hist = model.fit_generator(
#         Datagen(train_inputs, train_targets, batch_size, is_train=True),
#         steps_per_epoch = len(train_inputs) // batch_size,
#         epochs = n_epoch,
#         validation_data=Datagen(val_inputs, val_targets, batch_size),
#         validation_steps = len(val_inputs) // batch_size,
#         callbacks = [lr_schedule, macroF1(model, val_inputs, val_targets)],
#         shuffle = False,
#         verbose = 1
#         )
#     return hist


# def lrs(epoch):
#     if epoch<35:
#         lr = learning_rate
#     elif epoch<50:
#         lr = learning_rate/10
#     elif epoch<70:
#         lr = learning_rate/100
#     else:
#         lr = learning_rate/1000
#     return lr



In [ ]:
# def cbr(x, out_layer, kernel, stride, dilation):
#     x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     return x

# def se_block(x_in, layer_n):
#     x = GlobalAveragePooling1D()(x_in)
#     x = Dense(layer_n//8, activation="relu")(x)
#     x = Dense(layer_n, activation="sigmoid")(x)
#     x_out=Multiply()([x_in, x])
#     return x_out

# def resblock(x_in, layer_n, kernel, dilation, use_se=True):
#     x = cbr(x_in, layer_n, kernel, 1, dilation)
#     x = cbr(x, layer_n, kernel, 1, dilation)
#     if use_se:
#         x = se_block(x, layer_n)
#     x = Add()([x_in, x])
#     return x  

# def UUnet(input_shape=(None,1)):
#     layer_n = 64
#     kernel_size = 7
#     depth = 2

#     input_layer = Input(input_shape)    
#     input_layer_1 = AveragePooling1D(5)(input_layer)
#     input_layer_2 = AveragePooling1D(25)(input_layer)
    
#     ##########################
#     ### First U-Net: Big-U ###
#     ##########################
    
#     ########## Encoder 1
#     x = cbr(input_layer, layer_n, kernel_size, 1, 1)#1000
#     for i in range(depth):
#         x = resblock(x, layer_n, kernel_size, 1)
#     out_0 = x

#     x = cbr(x, layer_n*2, kernel_size, 5, 1)
#     for i in range(depth):
#         x = resblock(x, layer_n*2, kernel_size, 1)
#     out_1 = x

#     x = Concatenate()([x, input_layer_1])    
#     x = cbr(x, layer_n*3, kernel_size, 5, 1)
#     for i in range(depth):
#         x = resblock(x, layer_n*3, kernel_size, 1)
#     out_2 = x

#     x = Concatenate()([x, input_layer_2])    
#     x = cbr(x, layer_n*4, kernel_size, 5, 1)
#     for i in range(depth):
#         x = resblock(x, layer_n*4, kernel_size, 1)
    
#     ########### Decoder 1
#     x = UpSampling1D(5)(x)
#     x = Concatenate()([x, out_2])
#     x = cbr(x, layer_n*3, kernel_size, 1, 1)

#     x = UpSampling1D(5)(x)
#     x = Concatenate()([x, out_1])
#     x = cbr(x, layer_n*2, kernel_size, 1, 1)

#     x = UpSampling1D(5)(x)
#     x = Concatenate()([x, out_0])
#     x = cbr(x, layer_n, kernel_size, 1, 1)   
    
#     #############################
#     ### Second U-Net: Small-U ###
#     #############################
    
#     ########## Encoder 2
#     #x = cbr(input_layer, layer_n, kernel_size, 1, 1)#1000
#     for i in range(depth):
#         x = resblock(x, layer_n, kernel_size, 1)
#     out_0 = x

#     x = cbr(x, layer_n*2, kernel_size, 5, 1)
#     for i in range(depth):
#         x = resblock(x, layer_n*2, kernel_size, 1)
#     out_1 = x

#     x = Concatenate()([x, input_layer_1])    
#     x = cbr(x, layer_n*3, kernel_size, 5, 1)
#     for i in range(depth):
#         x = resblock(x, layer_n*3, kernel_size, 1)
    
#     ########### Decoder 2
#     x = UpSampling1D(5)(x)
#     x = Concatenate()([x, out_1])
#     x = cbr(x, layer_n*2, kernel_size, 1, 1)

#     x = UpSampling1D(5)(x)
#     x = Concatenate()([x, out_0])
#     x = cbr(x, layer_n, kernel_size, 1, 1) 
    
#     ###############################
#     ### Third U-Net: Smallest-U ###
#     ###############################
    
#     ########## Encoder 2
#     #x = cbr(input_layer, layer_n, kernel_size, 1, 1)#1000
#     for i in range(depth):
#         x = resblock(x, layer_n, kernel_size, 1)
#     out_0 = x
    
#     x = cbr(x, layer_n*2, kernel_size, 5, 1)
#     for i in range(depth):
#         x = resblock(x, layer_n*2, kernel_size, 1)
    
#     ########### Decoder 2
#     x = UpSampling1D(5)(x)
#     x = Concatenate()([x, out_0])
#     x = cbr(x, layer_n, kernel_size, 1, 1)
    
#     #classifier
#     x = Conv1D(11, kernel_size=kernel_size, strides=1, padding="same")(x)
#     out = Activation("softmax")(x)
    
#     model = Model(input_layer, out)
    
#     return model



# def augmentations(input_data, target_data):
#     #flip
#     if np.random.rand()<0.5:    
#         input_data = input_data[::-1]
#         target_data = target_data[::-1]

#     return input_data, target_data


# def Datagen(input_dataset, target_dataset, batch_size, is_train=False):
#     x=[]
#     y=[]
  
#     count=0
#     idx_1 = np.arange(len(input_dataset))
#     #idx_2 = np.arange(len(input_dataset))
#     np.random.shuffle(idx_1)
#     #np.random.shuffle(idx_2)

#     while True:
#         for i in range(len(input_dataset)):
#             input_data = input_dataset[idx_1[i]]
#             target_data = target_dataset[idx_1[i]]
#             #input_data_mix = input_dataset[idx_2[i]]
#             #target_data_mix = target_dataset[idx_2[i]]

#             if is_train:
#                 input_data, target_data = augmentations(input_data, target_data)
#                 #input_data_mix, target_data_mix = augmentations(input_data_mix, target_data_mix)
                
#             x.append(input_data)
#             y.append(target_data)
#             count+=1
#             if count==batch_size:
#                 x=np.array(x, dtype=np.float32)
#                 y=np.array(y, dtype=np.float32)
#                 inputs = x
#                 targets = y       
#                 x = []
#                 y = []
#                 count=0
#                 yield inputs, targets

# class macroF1(Callback):
#     def __init__(self, model, inputs, targets):
#         self.model = model
#         self.inputs = inputs
#         self.targets = np.argmax(targets, axis=2).reshape(-1)

#     def on_epoch_end(self, epoch, logs):
#         pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
#         f1_val = f1_score(self.targets, pred, average="macro")
#         print("val_f1_macro_score: ", f1_val)
        
# def model_fit(model, train_inputs, train_targets, val_inputs, val_targets, n_epoch, batch_size=32):
#     hist = model.fit_generator(
#         Datagen(train_inputs, train_targets, batch_size, is_train=True),
#         steps_per_epoch = len(train_inputs) // batch_size,
#         epochs = n_epoch,
#         validation_data=Datagen(val_inputs, val_targets, batch_size),
#         validation_steps = len(val_inputs) // batch_size,
#         callbacks = [lr_schedule, macroF1(model, val_inputs, val_targets)],
#         shuffle = False,
#         verbose = 1
#         )
#     return hist


# def lrs(epoch):
#     if epoch<30:
#         lr = learning_rate
#     elif epoch<50:
#         lr = learning_rate/10
#     elif epoch<70:
#         lr = learning_rate/100
#     elif epoch<110:
#         lr = learning_rate/1000
#     elif epoch<170:
#         lr = learning_rate/10000
#     else:
#         lr = learning_rate/50000
#     return lr


In [ ]:
def cbr(x, out_layer, kernel, stride, dilation):
    x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def se_block(x_in, layer_n):
    x = GlobalAveragePooling1D()(x_in)
    x = Dense(layer_n//8, activation="relu")(x)
    x = Dense(layer_n, activation="sigmoid")(x)
    x_out=Multiply()([x_in, x])
    return x_out

def resblock(x_in, layer_n, kernel, dilation, use_se=True):
    x = cbr(x_in, layer_n, kernel, 1, dilation)
    x = cbr(x, layer_n, kernel, 1, dilation)
    if use_se:
        x = se_block(x, layer_n)
    x = Add()([x_in, x])
    return x  

def UUnet(input_shape=(None,1)):
    layer_n = 64
    kernel_size1 = 7
    kernel_size2 = 5
    kernel_size3 = 3
    depth = 3

    input_layer = Input(input_shape)    
    input_layer_1 = AveragePooling1D(5)(input_layer)
    input_layer_2 = AveragePooling1D(25)(input_layer)
    
    ##########################
    ### First U-Net: Big-U ###
    ##########################
    
    ########## Encoder 1
    x = cbr(input_layer, layer_n, kernel_size1, 1, 1)#1000
    for i in range(depth):
        x = resblock(x, layer_n, kernel_size1, 1)
    out_0 = x

    x = cbr(x, layer_n*2, kernel_size2, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*2, kernel_size2, 1)
    out_1 = x

    x = Concatenate()([x, input_layer_1])    
    x = cbr(x, layer_n*3, kernel_size3, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*3, kernel_size3, 1)
    out_2 = x

    x = Concatenate()([x, input_layer_2])    
    x = cbr(x, layer_n*4, kernel_size3, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*4, kernel_size3, 1)
    
    ########### Decoder 1
    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_2])
    x = cbr(x, layer_n*3, kernel_size3, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_1])
    x = cbr(x, layer_n*2, kernel_size2, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_0])
    x = cbr(x, layer_n, kernel_size1, 1, 1)   
    
    #############################
    ### Second U-Net: Small-U ###
    #############################
    
    ########## Encoder 2
    #x = cbr(input_layer, layer_n, kernel_size, 1, 1)#1000
    for i in range(depth):
        x = resblock(x, layer_n, kernel_size1, 1)
    out_0 = x

    x = cbr(x, layer_n*2, kernel_size2, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*2, kernel_size2, 1)
    out_1 = x

    x = Concatenate()([x, input_layer_1])    
    x = cbr(x, layer_n*3, kernel_size3, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*3, kernel_size3, 1)
    
    ########### Decoder 2
    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_1])
    x = cbr(x, layer_n*2, kernel_size3, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_0])
    x = cbr(x, layer_n, kernel_size2, 1, 1) 
    
    ###############################
    ### Third U-Net: Small-U ###
    ###############################
    
    ########## Encoder 3
    #x = cbr(input_layer, layer_n, kernel_size, 1, 1)#1000
    for i in range(depth):
        x = resblock(x, layer_n, kernel_size1, 1)
    out_0 = x

    x = cbr(x, layer_n*2, kernel_size2, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*2, kernel_size2, 1)
    out_1 = x

    x = Concatenate()([x, input_layer_1])    
    x = cbr(x, layer_n*3, kernel_size3, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*3, kernel_size3, 1)
    
    ########### Decoder 2
    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_1])
    x = cbr(x, layer_n*2, kernel_size3, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_0])
    x = cbr(x, layer_n, kernel_size2, 1, 1) 
    
    ###############################
    ### Forth U-Net: big-U ###
    ###############################
    
    ########## Encoder 4
    x = cbr(input_layer, layer_n, kernel_size1, 1, 1)#1000
    for i in range(depth):
        x = resblock(x, layer_n, kernel_size1, 1)
    out_0 = x

    x = cbr(x, layer_n*2, kernel_size2, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*2, kernel_size2, 1)
    out_1 = x

    x = Concatenate()([x, input_layer_1])    
    x = cbr(x, layer_n*3, kernel_size3, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*3, kernel_size3, 1)
    out_2 = x

    x = Concatenate()([x, input_layer_2])    
    x = cbr(x, layer_n*4, kernel_size3, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n*4, kernel_size3, 1)
    
    ########### Decoder 4
    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_2])
    x = cbr(x, layer_n*3, kernel_size3, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_1])
    x = cbr(x, layer_n*2, kernel_size2, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_0])
    x = cbr(x, layer_n, kernel_size1, 1, 1)  
    
    #classifier
    x = Conv1D(11, kernel_size=kernel_size2, strides=1, padding="same")(x)
    out = Activation("softmax")(x)
    
    model = Model(input_layer, out)
    
    return model



def augmentations(input_data, target_data):
    #flip
    if np.random.rand()<0.5:    
        input_data = input_data[::-1]
        target_data = target_data[::-1]

    return input_data, target_data


def Datagen(input_dataset, target_dataset, batch_size, is_train=False):
    x=[]
    y=[]
  
    count=0
    idx_1 = np.arange(len(input_dataset))
    #idx_2 = np.arange(len(input_dataset))
    np.random.shuffle(idx_1)
    #np.random.shuffle(idx_2)

    while True:
        for i in range(len(input_dataset)):
            input_data = input_dataset[idx_1[i]]
            target_data = target_dataset[idx_1[i]]
            #input_data_mix = input_dataset[idx_2[i]]
            #target_data_mix = target_dataset[idx_2[i]]

            if is_train:
                input_data, target_data = augmentations(input_data, target_data)
                #input_data_mix, target_data_mix = augmentations(input_data_mix, target_data_mix)
                
            x.append(input_data)
            y.append(target_data)
            count+=1
            if count==batch_size:
                x=np.array(x, dtype=np.float32)
                y=np.array(y, dtype=np.float32)
                inputs = x
                targets = y       
                x = []
                y = []
                count=0
                yield inputs, targets

class macroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)

    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        f1_val = f1_score(self.targets, pred, average="macro")
        print("val_f1_macro_score: ", f1_val)
        
def model_fit(model, train_inputs, train_targets, val_inputs, val_targets, n_epoch, batch_size=32):
    hist = model.fit_generator(
        Datagen(train_inputs, train_targets, batch_size, is_train=True),
        steps_per_epoch = len(train_inputs) // batch_size,
        epochs = n_epoch,
        validation_data=Datagen(val_inputs, val_targets, batch_size),
        validation_steps = len(val_inputs) // batch_size,
        callbacks = [lr_schedule, macroF1(model, val_inputs, val_targets)],
        shuffle = False,
        verbose = 1
        )
    return hist


def lrs(epoch):
    if epoch<45:
        lr = learning_rate
    elif epoch<75:
        lr = learning_rate/10
    elif epoch<115:
        lr = learning_rate/100
    elif epoch<165:
        lr = learning_rate/1000
#     elif epoch<170:
#         lr = learning_rate/10000
    else:
        lr = learning_rate/10000
    return lr


In [ ]:
# def conv3x3(x, out_filters, strides=1):
#     x = Conv1D(out_filters, 3, padding='same', strides=strides, use_bias=False, kernel_initializer='he_normal')(x)
#     return x


# def basic_Block(input, out_filters, strides=1, with_conv_shortcut=False):
#     x = conv3x3(input, out_filters, strides)
#     x = BatchNormalization(axis=-1)(x)
#     x = Activation('relu')(x)

#     x = conv3x3(x, out_filters)
#     x = BatchNormalization(axis=-1)(x)

#     if with_conv_shortcut:
#         residual = Conv1D(out_filters, 1, strides=strides, use_bias=False, kernel_initializer='he_normal')(input)
#         residual = BatchNormalization(axis=-1)(residual)
#         x = add([x, residual])
#     else:
#         x = add([x, input])

#     x = Activation('relu')(x)
#     return x


# def bottleneck_Block(input, out_filters, strides=1, with_conv_shortcut=False):
#     expansion = 4
#     de_filters = int(out_filters / expansion)

#     x = Conv1D(de_filters, 1, use_bias=False, kernel_initializer='he_normal')(input)
#     x = BatchNormalization(axis=-1)(x)
#     x = Activation('relu')(x)

#     x = Conv1D(de_filters, 3, strides=strides, padding='same', use_bias=False, kernel_initializer='he_normal')(x)
#     x = BatchNormalization(axis=-1)(x)
#     x = Activation('relu')(x)

#     x = Conv1D(out_filters, 1, use_bias=False, kernel_initializer='he_normal')(x)
#     x = BatchNormalization(axis=-1)(x)

#     if with_conv_shortcut:
#         residual = Conv1D(out_filters, 1, strides=strides, use_bias=False, kernel_initializer='he_normal')(input)
#         residual = BatchNormalization(axis=-1)(residual)
#         x = add([x, residual])
#     else:
#         x = add([x, input])

#     x = Activation('relu')(x)
#     return x


# def stem_net(input):
#     x = Conv1D(64, 3, strides=2, padding='same', use_bias=False, kernel_initializer='he_normal')(input)
#     x = BatchNormalization(axis=-1)(x)
#     x = Activation('relu')(x)

#     # x = Conv2D(64, 3, strides=(2, 2), padding='same', use_bias=False, kernel_initializer='he_normal')(x)
#     # x = BatchNormalization(axis=3)(x)
#     # x = Activation('relu')(x)

#     x = bottleneck_Block(x, 256, with_conv_shortcut=True)
#     x = bottleneck_Block(x, 256, with_conv_shortcut=False)
#     x = bottleneck_Block(x, 256, with_conv_shortcut=False)
#     x = bottleneck_Block(x, 256, with_conv_shortcut=False)

#     return x


# def transition_layer1(x, out_filters_list=[32, 64]):
#     x0 = Conv1D(out_filters_list[0], 3, padding='same', use_bias=False, kernel_initializer='he_normal')(x)
#     x0 = BatchNormalization(axis=-1)(x0)
#     x0 = Activation('relu')(x0)

#     x1 = Conv1D(out_filters_list[1], 3, strides=2,
#                 padding='same', use_bias=False, kernel_initializer='he_normal')(x)
#     x1 = BatchNormalization(axis=-1)(x1)
#     x1 = Activation('relu')(x1)

#     return [x0, x1]


# def make_branch1_0(x, out_filters=32):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def make_branch1_1(x, out_filters=64):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def fuse_layer1(x):
#     x0_0 = x[0]
#     x0_1 = Conv1D(32, 1, use_bias=False, kernel_initializer='he_normal')(x[1])
#     x0_1 = BatchNormalization(axis=-1)(x0_1)
#     x0_1 = UpSampling1D(size=2)(x0_1)
#     x0 = add([x0_0, x0_1])

#     x1_0 = Conv1D(64, 3, strides=2, padding='same', use_bias=False, kernel_initializer='he_normal')(x[0])
#     x1_0 = BatchNormalization(axis=-1)(x1_0)
#     x1_1 = x[1]
#     x1 = add([x1_0, x1_1])
#     return [x0, x1]


# def transition_layer2(x, out_filters_list=[32, 64, 128]):
#     x0 = Conv1D(out_filters_list[0], 3, padding='same', use_bias=False, kernel_initializer='he_normal')(x[0])
#     x0 = BatchNormalization(axis=-1)(x0)
#     x0 = Activation('relu')(x0)

#     x1 = Conv1D(out_filters_list[1], 3, padding='same', use_bias=False, kernel_initializer='he_normal')(x[1])
#     x1 = BatchNormalization(axis=-1)(x1)
#     x1 = Activation('relu')(x1)

#     x2 = Conv1D(out_filters_list[2], 3, strides=2,
#                 padding='same', use_bias=False, kernel_initializer='he_normal')(x[1])
#     x2 = BatchNormalization(axis=-1)(x2)
#     x2 = Activation('relu')(x2)

#     return [x0, x1, x2]


# def make_branch2_0(x, out_filters=32):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def make_branch2_1(x, out_filters=64):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def make_branch2_2(x, out_filters=128):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def fuse_layer2(x):
#     x0_0 = x[0]
#     x0_1 = Conv1D(32, 1, use_bias=False, kernel_initializer='he_normal')(x[1])
#     x0_1 = BatchNormalization(axis=-1)(x0_1)
#     x0_1 = UpSampling1D(size=2)(x0_1)
#     x0_2 = Conv1D(32, 1, use_bias=False, kernel_initializer='he_normal')(x[2])
#     x0_2 = BatchNormalization(axis=-1)(x0_2)
#     x0_2 = UpSampling1D(size=4)(x0_2)
#     x0 = add([x0_0, x0_1, x0_2])

#     x1_0 = Conv1D(64, 3, strides=2, padding='same', use_bias=False, kernel_initializer='he_normal')(x[0])
#     x1_0 = BatchNormalization(axis=-1)(x1_0)
#     x1_1 = x[1]
#     x1_2 = Conv1D(64, 1, use_bias=False, kernel_initializer='he_normal')(x[2])
#     x1_2 = BatchNormalization(axis=-1)(x1_2)
#     x1_2 = UpSampling1D(size=2)(x1_2)
#     x1 = add([x1_0, x1_1, x1_2])

#     x2_0 = Conv1D(32, 3, strides=2, padding='same', use_bias=False, kernel_initializer='he_normal')(x[0])
#     x2_0 = BatchNormalization(axis=-1)(x2_0)
#     x2_0 = Activation('relu')(x2_0)
#     x2_0 = Conv1D(128, 3, strides=2, padding='same', use_bias=False, kernel_initializer='he_normal')(x2_0)
#     x2_0 = BatchNormalization(axis=-1)(x2_0)
#     x2_1 = Conv1D(128, 3, strides=2, padding='same', use_bias=False, kernel_initializer='he_normal')(x[1])
#     x2_1 = BatchNormalization(axis=-1)(x2_1)
#     x2_2 = x[2]
#     x2 = add([x2_0, x2_1, x2_2])
#     return [x0, x1, x2]


# def transition_layer3(x, out_filters_list=[32, 64, 128, 256]):
#     x0 = Conv1D(out_filters_list[0], 3, padding='same', use_bias=False, kernel_initializer='he_normal')(x[0])
#     x0 = BatchNormalization(axis=-1)(x0)
#     x0 = Activation('relu')(x0)

#     x1 = Conv1D(out_filters_list[1], 3, padding='same', use_bias=False, kernel_initializer='he_normal')(x[1])
#     x1 = BatchNormalization(axis=-1)(x1)
#     x1 = Activation('relu')(x1)

#     x2 = Conv1D(out_filters_list[2], 3, padding='same', use_bias=False, kernel_initializer='he_normal')(x[2])
#     x2 = BatchNormalization(axis=-1)(x2)
#     x2 = Activation('relu')(x2)

#     x3 = Conv1D(out_filters_list[3], 3, strides=2,
#                 padding='same', use_bias=False, kernel_initializer='he_normal')(x[2])
#     x3 = BatchNormalization(axis=-1)(x3)
#     x3 = Activation('relu')(x3)

#     return [x0, x1, x2, x3]


# def make_branch3_0(x, out_filters=32):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def make_branch3_1(x, out_filters=64):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def make_branch3_2(x, out_filters=128):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def make_branch3_3(x, out_filters=256):
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     x = basic_Block(x, out_filters, with_conv_shortcut=False)
#     return x


# def fuse_layer3(x):
#     x0_0 = x[0]
#     x0_1 = Conv1D(32, 1, use_bias=False, kernel_initializer='he_normal')(x[1])
#     x0_1 = BatchNormalization(axis=-1)(x0_1)
#     x0_1 = UpSampling1D(size=2)(x0_1)
#     x0_2 = Conv1D(32, 1, use_bias=False, kernel_initializer='he_normal')(x[2])
#     x0_2 = BatchNormalization(axis=-1)(x0_2)
#     x0_2 = UpSampling1D(size=4)(x0_2)
#     x0_3 = Conv1D(32, 1, use_bias=False, kernel_initializer='he_normal')(x[3])
#     x0_3 = BatchNormalization(axis=-1)(x0_3)
#     x0_3 = UpSampling1D(size=8)(x0_3)
#     x0 = concatenate([x0_0, x0_1, x0_2, x0_3], axis=-1)
#     return x0


# def final_layer(x, classes=11):
#     x = UpSampling1D(size=2)(x)
#     x = Conv1D(classes, 1, use_bias=False, kernel_initializer='he_normal')(x)
#     x = BatchNormalization(axis=-1)(x)
#     x = Activation('softmax', name='Classification')(x)
#     return x


# def seg_hrnet(input_shape=(None,1)):

#     inputs = Input(input_shape)

#     x = stem_net(inputs)

#     x = transition_layer1(x)
#     x0 = make_branch1_0(x[0])
#     x1 = make_branch1_1(x[1])
#     x = fuse_layer1([x0, x1])

#     x = transition_layer2(x)
#     x0 = make_branch2_0(x[0])
#     x1 = make_branch2_1(x[1])
#     x2 = make_branch2_2(x[2])
#     x = fuse_layer2([x0, x1, x2])

#     x = transition_layer3(x)
#     x0 = make_branch3_0(x[0])
#     x1 = make_branch3_1(x[1])
#     x2 = make_branch3_2(x[2])
#     x3 = make_branch3_3(x[3])
#     x = fuse_layer3([x0, x1, x2, x3])

#     out = final_layer(x, classes=11)

#     model = Model(inputs=inputs, outputs=out)

#     return model


# def augmentations(input_data, target_data):
#     #flip
#     if np.random.rand()<0.5:    
#         input_data = input_data[::-1]
#         target_data = target_data[::-1]

#     return input_data, target_data


# def Datagen(input_dataset, target_dataset, batch_size, is_train=False):
#     x=[]
#     y=[]
  
#     count=0
#     idx_1 = np.arange(len(input_dataset))
#     #idx_2 = np.arange(len(input_dataset))
#     np.random.shuffle(idx_1)
#     #np.random.shuffle(idx_2)

#     while True:
#         for i in range(len(input_dataset)):
#             input_data = input_dataset[idx_1[i]]
#             target_data = target_dataset[idx_1[i]]
#             #input_data_mix = input_dataset[idx_2[i]]
#             #target_data_mix = target_dataset[idx_2[i]]

#             if is_train:
#                 input_data, target_data = augmentations(input_data, target_data)
#                 #input_data_mix, target_data_mix = augmentations(input_data_mix, target_data_mix)
                
#             x.append(input_data)
#             y.append(target_data)
#             count+=1
#             if count==batch_size:
#                 x=np.array(x, dtype=np.float32)
#                 y=np.array(y, dtype=np.float32)
#                 inputs = x
#                 targets = y       
#                 x = []
#                 y = []
#                 count=0
#                 yield inputs, targets

# class macroF1(Callback):
#     def __init__(self, model, inputs, targets):
#         self.model = model
#         self.inputs = inputs
#         self.targets = np.argmax(targets, axis=2).reshape(-1)

#     def on_epoch_end(self, epoch, logs):
#         pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
#         f1_val = f1_score(self.targets, pred, average="macro")
#         print("val_f1_macro_score: ", f1_val)
        
# def model_fit(model, train_inputs, train_targets, val_inputs, val_targets, n_epoch, batch_size=32):
#     hist = model.fit_generator(
#         Datagen(train_inputs, train_targets, batch_size, is_train=True),
#         steps_per_epoch = len(train_inputs) // batch_size,
#         epochs = n_epoch,
#         validation_data=Datagen(val_inputs, val_targets, batch_size),
#         validation_steps = len(val_inputs) // batch_size,
#         callbacks = [lr_schedule, macroF1(model, val_inputs, val_targets)],
#         shuffle = False,
#         verbose = 1
#         )
#     return hist


# def lrs(epoch):
#     if epoch<35:
#         lr = learning_rate
#     elif epoch<50:
#         lr = learning_rate/100
#     elif epoch<90:
#         lr = learning_rate/1000
#     elif epoch<150:
#         lr = learning_rate/10000
#     elif epoch<230:
#         lr = learning_rate/100000
#     else:
#         lr = learning_rate/1000000
#     return lr



## Training

In [ ]:
K.clear_session()
# model = DenseHRNet()
model = UUnet()
# print(model.summary())

learning_rate=0.0005
n_epoch=200
# batch_size=100
batch_size=8

# earlystopping = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
lr_schedule = LearningRateScheduler(lrs)
model.compile(loss=categorical_crossentropy, 
              optimizer=Adam(lr=learning_rate),
              metrics=["accuracy"])

hist = model_fit(model, train_input, train_target, val_input, val_target, n_epoch, batch_size)

In [ ]:
# pred = ((model.predict(val_input)+model.predict(val_input[:,::-1,:])[:,::-1,:])/2).reshape(-1)
# print("VALIDATION_SCORE: ", cohen_kappa_score(val_target.reshape(-1), np.round(pred, 0), weights="quadratic"))
pred = np.argmax((model.predict(val_input)+model.predict(val_input[:,::-1,:])[:,::-1,:])/2, axis=2).reshape(-1)
gt = np.argmax(val_target, axis=2).reshape(-1)
print("SCORE_oldmetric: ", cohen_kappa_score(gt, pred, weights="quadratic"))
print("SCORE_newmetric: ", f1_score(gt, pred, average="macro"))

## Predict and Submit
This is not the main topic of this kernel, so I just round predicted values.

In [ ]:
pred = np.argmax((model.predict(test_input)+model.predict(test_input[:,::-1,:])[:,::-1,:])/2, axis=2).reshape(-1)

df_sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})
df_sub.open_channels = np.array(np.round(pred,0), np.int)
df_sub.to_csv("submission.csv",index=False)